In [ ]:
!nvidia-smi

In [ ]:
import os

import numpy as np
from keras.optimizers import Adam
from keras.callbacks import Callback, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from PIL import Image

from unet import get_unet, jaccard_coef_int, jaccard_coef_loss

import matplotlib.pyplot as plt

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

## Create model

In [ ]:
input_shape = (512, 512, 3)
batch_size = 8

In [ ]:
model = get_unet(3, input_shape[0], input_shape[1], classes=1)
# model.load_weights(weights_path)

## Load data

In [ ]:
import glob
import cv2
import numpy as np
import random
import math
import json
SEED = 448
import matplotlib.pyplot as plt
from imgaug import augmenters as iaa
import shutil
import skimage.io as io
from skimage.transform import resize
from collections import Counter
from PIL import Image, ImageDraw

In [ ]:
freck = json.load(open('./labels/all.json'))

In [ ]:
# good[0]['Label']['Head'][0]["geometry"]

In [ ]:
# polygon = [(g["x"], g["y"]) for g in good[0]['Label']['Head'][0]["geometry"]]
# img = Image.new('L', (668, 551), 0)
# ImageDraw.Draw(img).polygon(polygon, outline=1, fill=1)
# mask = np.array(img)
# head_mask = resize(mask, (512, 512))
# plt.imshow(head_mask)
# plt.show()

In [ ]:
def generator(images, BATCH_SIZE, input_shape):
    i = 0
    img_size = input_shape[0]
    while True:
        x_batch = np.empty((BATCH_SIZE, input_shape[0], input_shape[1], input_shape[2])) #, dtype=np.uint8)
        y_batch = np.empty((BATCH_SIZE, input_shape[0], input_shape[1], 1), dtype=np.uint8)
        for j in range(BATCH_SIZE):
            randomimg = images[np.random.randint(len(images))]
            imgpath = randomimg["local_path"]
            x1, y1, x2, y2 = randomimg["bbox"]
            img = io.imread(imgpath)[y1:y2, x1:x2]

            xscale = float(input_shape[0]) / img.shape[0] 
            yscale = float(input_shape[1]) / img.shape[1] 
            
            imgresized = resize(img, (input_shape[0], input_shape[1]))            

            mask = np.zeros((input_shape[0], input_shape[1]))
            for freckle in randomimg["freckles"]:
                y = int((freckle[0]-x1) * yscale)
                x = int((freckle[1]-y1) * xscale)
                mask[x, y] = 1
                maskdil = cv2.dilate(mask, np.ones((5, 5)))
            x_batch[j, ...] = imgresized
            y_batch[j, ..., 0] = maskdil
            
        yield x_batch, y_batch

In [ ]:
gen = generator(freck, batch_size, input_shape)

In [ ]:
xb, yb = next(gen)

In [ ]:
for i in range(batch_size):
    plt.figure(figsize=(12, 12))
    plt.imshow(xb[i, ...])
    plt.imshow(yb[i, ...].squeeze(), alpha=0.5)
    plt.show()

## Train

In [ ]:
lr=0.001
adam = Adam(lr=lr)
model.compile(adam, loss='binary_crossentropy', metrics=['binary_crossentropy'])

In [ ]:
history = model.fit_generator(
        generator=gen,
        steps_per_epoch=50,
        epochs=100,
        verbose=1)

In [ ]:
model

In [ ]:
img = resize(io.imread('/root/data/reidentification/heads_with_eye/erko-rotoy/left_erko-rotoy_1_1539164547711.jpg_161_128.jpg'), (512, 512))

In [ ]:
# thresh = 0.001
# pred[pred> thresh] =1
# pred[pred< thresh] = 0

In [ ]:
images= glob.glob("/root/data/reidentification/heads/*")

In [ ]:
random.shuffle(images)

In [ ]:
xb, yb = next(gen)

In [ ]:
pred = model.predict(xb)

In [ ]:
for i in range(batch_size):
    plt.figure(figsize=(8, 8))
    plt.imshow(xb[i, ...])
    plt.imshow(pred[i, ...].squeeze(), alpha=0.5)
    plt.show()

In [ ]:
# f, ax = plt.subplots(2, 2, figsize=(10, 15))
# for (i, imgpath) in enumerate(images[:4]):
#     img= resize(io.imread(imgpath), (512, 512))
#     pred = model.predict(np.expand_dims(img, axis=0))
#     print(pred.max())
#     x = i // 2
#     y = i % 2
#     ax[x,y].imshow(img)
#     ax[x,y].imshow(pred.squeeze(), alpha=0.5)
# plt.tight_layout()
# plt.show()

In [ ]:
plt.imshow(pred.squeeze())

In [ ]:
plt.imshow(pred.squeeze()* head_mask) # , alpha=0.5)

In [ ]:
k = pred.squeeze()* head_mask
plt.imshow(k[200:300, 200:300])
plt.colorbar()